<a href="https://colab.research.google.com/github/22053604/Cardiovascular-Health-Analysis/blob/main/Development_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Version Control**

In [1]:
#import necessary libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [6]:
#Connect to the SQLite database
conn = sqlite3.connect('cardiohealth.db')
cursor = conn.cursor()

In [7]:
#Retrieve and display table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

#List all tables
print(cursor.fetchall())

[('medical_examination',)]


In [8]:
#Replacing with actual table name
query = """
SELECT age, gender, height, weight, ap_hi, ap_lo, cholesterol, gluc, smoke, alco, active, cardio
FROM medical_examination;
"""

#Load SQL result into a Panadas dataframe for analysis
query = "SELECT * FROM medical_examination;"
df = pd.read_sql_query(query, conn)

In [ ]:
#Clean the data
#Show the missing values
df.isnull().sum()